# Пример для сабмита решения
Данные содержат сведения о спросе на отели. Датасет содержит 36 275 наблюдений. Каждое наблюдение представляет собой бронирование отеля. Набор данных учитывает бронирования, которые должны прибыть в период с 2017 года по 2018 год, включая фактически поступившие и отмененные бронирования.

Данные содержат сведения бронированиях в одном отеле.
- Тренировочный набор - **24304 строк**.
- Тестовый набор - **11971 строк**.

* **id**: Уникальный идентификатор бронирования
* **no_of_adults**: Число взрослых гостей
* **no_of_children**: Число детей среди гостей
* **no_of_weekend_nights**: Число ночей, приходящихся на выходные дни (суббота или воскресенье), которые гость забронировал
* **no_of_week_nights**: Число ночей, приходящихся на будние дни (с понедельника по пятницу), которые гость забронировал
* **type_of_meal_plan**: Тип пансиона, который выбрал гость
* **required_car_parking_space**: Нужно ли место для парковки автомобиля гостю? (0 - Нет, 1- Да)
* **room_type_reserved**: Тип забронированной комнаты. Реальные значения уже закодированы.
* **lead_time**: Число дней между днем прибытия гостя в страну и началом бронирования
* **arrival_year**: Год запланированного начала бронирования
* **arrival_month**: Месяц запланированного начала бронирования
* **arrival_date**: Дата запланированного начала бронирования
* **market_segment_type**: Способ бронирования (онлайн/оффлайн)
* **repeated_guest**: Был ли гость уже в отеле до этого бронирования? (0 - Нет, 1 - Да)
* **no_of_previous_cancellations**: Число бронирований, которые гость уже отменял до данного бронирования
* **no_of_previous_bookings_not_canceled**: Число бронирований, которые гость не были отменены до данного бронирования
* **avg_price_per_room**: Средняя стоимость бронирования комнаты за день; цены образуются динамически (в евро).
* **no_of_special_requests**: Общее число особенных пожеланий обозначенных гостем (т.е. верхний этаж, вид из окна номера и т.д.)

Целевая переменная:
- **booking_status**: 0 - Отмена или 1 - Гость приехал в отель.

In [1]:
# Иморитирование библиотек, которые вам скорее всего понадобятся
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import sklearn

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cancel-or-check-in/train.csv
/kaggle/input/cancel-or-check-in/test.csv


## Читаем данные

In [3]:
# Считываем тренировочные данные
train = pd.read_csv("/kaggle/input/cancel-or-check-in/train.csv")

print(f"Train dataset shape: {train.shape}")

Train dataset shape: (24304, 19)


In [4]:
# Считываем тестовые данные
test = pd.read_csv("/kaggle/input/cancel-or-check-in/test.csv")

print(f"Test dataset shape: {test.shape}")

Test dataset shape: (11971, 18)


In [5]:
# Посмотрим как выглядят данные
train.head()

,Booking_ID,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
0,INN13556,2,0,2,3,Not Selected,0,Room_Type 1,59,2018,5,21,Online,0,0,0,116.10,1,0
1,INN08357,2,0,2,0,Not Selected,0,Room_Type 1,30,2018,9,25,Online,0,0,0,89.10,0,0
2,INN31336,2,0,0,2,Meal Plan 1,0,Room_Type 1,19,2018,6,22,Corporate,0,0,0,100.00,0,1
3,INN10369,2,0,0,3,Not Selected,0,Room_Type 1,66,2018,3,16,Online,0,0,0,73.50,0,0
4,INN29656,2,0,2,3,Meal Plan 1,0,Room_Type 4,93,2018,4,17,Online,0,0,0,99.45,1,1


In [6]:
# Ваш код здесь

## Исследование данных

Прежде чем переходить к непосредственному построению модели, в первую очередь необходимо исследовать имеющихся у вас набор данных:
1. Какую задачу вы решаете (классификация бинарная или многоклассовая) и какая переменная является целевой?
2. Сколько переменных в данных и какого они типа?
3. Имеются ли в данных пропуски?
4. Сбалансированны ли классы?

In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24304 entries, 0 to 24303
Data columns (total 19 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Booking_ID                            24304 non-null  object 
 1   no_of_adults                          24304 non-null  int64  
 2   no_of_children                        24304 non-null  int64  
 3   no_of_weekend_nights                  24304 non-null  int64  
 4   no_of_week_nights                     24304 non-null  int64  
 5   type_of_meal_plan                     24304 non-null  object 
 6   required_car_parking_space            24304 non-null  int64  
 7   room_type_reserved                    24304 non-null  object 
 8   lead_time                             24304 non-null  int64  
 9   arrival_year                          24304 non-null  int64  
 10  arrival_month                         24304 non-null  int64  
 11  arrival_date   

In [8]:
# Ваш код здесь

## Предобработка данных

Исходя из анализа данных, определите, какие шаги нужно выполнить для обработки набора данных, что бы сделать его пригодным для ML-модели.

Подсказка:
* Заполнение пропусков в данных
* Кодирование категориальных признаков
* Стандартизация/нормирование численных признаков
* Выявление и устранение выбросов в данных

In [9]:
# Ваш код здесь

## Обучение модели
Теперь, когда данных готовы, приступите к выборку и обучению ML-модели.

In [10]:
# Ваш код здесь

## Предсказание ответа для тестового набора данных
В конечном итоге, запишите предсказания Вашей модели для тестовых данных. 

Пример записи и сабмита на лидерборд приведен ниже.

In [11]:
print(f"Test data of size: {test.shape}\n")
test.head()

Test data of size: (11971, 18)



,Booking_ID,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests
0,INN04969,2,1,1,0,Meal Plan 1,0,Room_Type 1,3,2017,8,23,Online,0,0,0,90.00,3
1,INN34541,2,0,1,2,Meal Plan 1,0,Room_Type 4,9,2018,2,12,Offline,0,0,0,48.67,0
2,INN36109,2,0,2,2,Meal Plan 1,0,Room_Type 1,24,2018,12,25,Online,0,0,0,95.20,1
3,INN01554,2,0,0,3,Meal Plan 1,0,Room_Type 1,23,2018,6,21,Online,0,0,0,127.67,0
4,INN24975,2,1,0,2,Meal Plan 1,0,Room_Type 4,9,2018,9,8,Online,0,0,0,201.50,2


In [12]:
# Предскажите целевую переменную для тестового набора
# y_pred_test = model.predict(test)

# А в примере сгененируем ее случайно из равномерного дискретного распределения
y_pred_test = np.random.randint(0, 2, size=(test.shape[0]), dtype=int)
print(y_pred_test.shape)

(11971,)


In [13]:
y_pred_test = pd.DataFrame(y_pred_test, columns=['is_canceled'])
y_pred_test = y_pred_test.reset_index()

y_pred_test.to_csv("solution.csv", index=False)